In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, Input, BatchNormalization
# from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint, Callback
from keras.optimizers import SGD, Adam
from keras.utils.training_utils import multi_gpu_model
from keras.preprocessing.image import ImageDataGenerator
from keras import initializers
import prepare_data as pd
import numpy
import math
import scipy.misc as spm

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from PIL import Image

from lr_multiplier import LearningRateMultiplier

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
# see example https://github.com/keras-team/keras/issues/8649
class MyCbk(Callback):

    def __init__(self, model):
         self.model_to_save = model

    def on_epoch_end(self, epoch, logs=None):
        self.model_to_save.save('checkpoints/64-9-3-5_128-64-SRCNN_model_at_epoch_%d.h5' % epoch)


In [3]:
def psnr(target, ref):
    # assume RGB image
    target_data = numpy.array(target, dtype=float)
    ref_data = numpy.array(ref, dtype=float)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = math.sqrt(numpy.mean(diff ** 2.))

    return 20 * math.log10(255. / rmse)

In [4]:
def model(optimizer='SGD', all_layers=True, processing='GPU', lr=0.0001):
    # lrelu = LeakyReLU(alpha=0.1)                                              
#     SRCNN = Sequential()                                                        
#     SRCNN.add(Conv2D(filters=128, kernel_size=(9, 9), init='glorot_uniform',  
#                      activation='relu', border_mode='valid', bias=True, input_shape=(32, 32, 1)))
#     SRCNN.add(Conv2D(filters=64, kernel_size=(3, 3), init='glorot_uniform',   
#                      activation='relu', border_mode='same', bias=True))         
#     # SRCNN.add(BatchNormalization())                                           
#     SRCNN.add(Conv2D(filters=1, kernel_size=(5, 5), init='glorot_uniform',    
#                      activation='linear', border_mode='valid', bias=True))      
#     adam = Adam(lr=0.0001)                                                     
# #     sgd = SGD(lr=0.0003)                                                        
#     SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])
#     return SRCNN 

    multipliers = {'conv2d_1': 1, 'conv2d_2': 1, 'conv2d_3': 0.1}
    
    optimizer = optimizer.upper()
    processing = processing.upper()
    
    print('Using {}.'.format(optimizer))

    if optimizer == 'SGD' and all_layers:
        opt = LearningRateMultiplier(SGD, lr_multipliers=multipliers, lr=lr)
    elif optimizer == 'SGD':
        opt = SGD(lr=0.0001)
    elif all_layers:
        opt = LearningRateMultiplier(Adam, lr_multipliers=multipliers, lr=lr)
    else:
        opt = Adam(lr=lr)
    # lrelu = LeakyReLU(alpha=0.1)
    
    SRCNN = Sequential()
    SRCNN.add(Conv2D(filters=128, kernel_size=(9, 9), 
                     kernel_initializer='glorot_uniform', bias_initializer='random_uniform',
                     activation='relu', padding='same', use_bias=True, input_shape=(64, 64, 1), name='conv2d_1'))

    SRCNN.add(Conv2D(filters=64, kernel_size=(3, 3), 
                     kernel_initializer='glorot_uniform', bias_initializer='random_uniform',
                     activation='relu', padding='same', use_bias=True, name='conv2d_2'))
    
    # SRCNN.add(BatchNormalization())
    SRCNN.add(Conv2D(filters=1, kernel_size=(5, 5),
                     kernel_initializer='glorot_uniform', bias_initializer='random_uniform',
                     activation='linear', padding='same', use_bias=True,  name='conv2d_3'))
    SRCNN_MULTI = multi_gpu_model(SRCNN, gpus=8)
    if processing == 'GPU':    
        SRCNN_MULTI.compile(optimizer=opt, loss='mean_squared_error', metrics=['mean_squared_error'])
    else:
        SRCNN.compile(optimizer=opt, loss='mean_squared_error', metrics=['mean_squared_error'])
    return SRCNN, SRCNN_MULTI

In [5]:
def predict_model():
    # lrelu = LeakyReLU(alpha=0.1)
    sgd = SGD(lr=0.0001)
    
    SRCNN = Sequential()
    SRCNN.add(Conv2D(filters=128, kernel_size=(9, 9), 
                     kernel_initializer='glorot_uniform', bias_initializer='random_uniform',
                     activation='relu', padding='valid', use_bias=True, input_shape=(None, None, 1), name='conv2d_1'))
        
    SRCNN.add(Conv2D(filters=64, kernel_size=(3, 3), 
                     kernel_initializer='glorot_uniform', bias_initializer='random_uniform',
                     activation='relu', padding='same', use_bias=True, name='conv2d_2'))
        
    # SRCNN.add(BatchNormalization())
    SRCNN.add(Conv2D(filters=1, kernel_size=(5, 5),
                     kernel_initializer='glorot_uniform', bias_initializer='random_uniform',
                     activation='linear', padding='valid', use_bias=True,  name='conv2d_3'))
    adam = Adam(lr=0.0003)
    sgd_last = SGD(lr=0.00001)

    SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])
    return SRCNN

In [6]:
def train(processing='GPU', optimizer='SGD', use_data_augmentation=False, epochs=200, lr=0.0001):
#     srcnn_model = model()
#     print(srcnn_model.summary())                                                
#     data, label = pd.read_training_data("./crop_train.h5")
#     val_data, val_label = pd.read_training_data("./test.h5")                    
                                                                                
#     checkpoint = ModelCheckpoint("SRCNN_check.h5", monitor='val_loss', verbose=1, save_best_only=True,
#                                  save_weights_only=False, mode='min')           
#     callbacks_list = [checkpoint]                                               
                                                                                
#     srcnn_model.fit(data, label, batch_size=1024,
#                     callbacks=callbacks_list, shuffle=True, nb_epoch=200, verbose=2, validation_split=0.2)

    optimizer = optimizer.upper()
    processing = processing.upper()
    
    srcnn_model, srcnn_multi_model = model(optimizer=optimizer, processing=processing, lr=lr)

    callbacks_list = []
    BS = 1024
    
    if processing == 'GPU' or use_data_augmentation:    
        print(srcnn_multi_model.summary())
        # see example https://github.com/keras-team/keras/issues/8649
        cbk = MyCbk(srcnn_model)
        
    else:
        print(srcnn_model.summary())
        checkpoint = ModelCheckpoint("64SRCNN_check.h5", monitor='val_loss', 
                                     verbose=2, 
                                     save_best_only=True,
                                     save_weights_only=False)
        callbacks_list.append(checkpoint)
        
    data, label = pd.read_training_data("./64crop_train.h5")
    val_data, val_label = pd.read_training_data("./64test.h5")

    max_value = data.max()

    data = data/max_value
    label = label/max_value

    val_data = val_data/max_value
    val_label = val_label/max_value

    
    # construct the training image generator for data augmentation
    datagen = ImageDataGenerator(rotation_range=45,
                                 zoom_range=0.15,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 horizontal_flip=True, 
                                 vertical_flip=True)

    # train the network

    if use_data_augmentation and processing == 'GPU':
        history = srcnn_multi_model.fit_generator(datagen.flow(data, label, batch_size=BS),
                                                    validation_data=(val_data, val_label),
                                                    steps_per_epoch=len(data) // BS,
                                                    epochs=epochs, 
                                                    verbose=2, 
                                                    shuffle=True,
                                                    callbacks=[cbk],
                                                    use_multiprocessing=True)
    elif processing == 'GPU':
        srcnn_multi_model.fit(data, label, batch_size=BS,
                              validation_split=0.2,
    #                       validation_data=(val_data, val_label),
                            shuffle=True, 
                            epochs=epochs, 
                            verbose=2,
                            callbacks=[cbk])
    elif use_data_augmentation:
        history = srcnn_model.fit_generator(datagen.flow(data, label, batch_size=BS),
                                          validation_data=(val_data, val_label),
                                          steps_per_epoch=len(data) // BS,
                                          epochs=epochs, 
                                          verbose=2, 
                                          shuffle=True,
                                          callbacks=[cbk])
    else:
        srcnn_model.fit(data, label, batch_size=BS,
                            validation_split=0.2,
    #                       validation_data=(val_data, val_label),
                            shuffle=True, 
                            epochs=epochs, 
                            verbose=2,
                            callbacks=callbacks_list)

    # srcnn_model.load_weights("m_model_adam.h5")

In [7]:
data, label = pd.read_training_data("./64crop_train.h5")
val_data, val_label = pd.read_training_data("./64test.h5")

max_value = data.max()

data = data/max_value
label = label/max_value

val_data = val_data/max_value
val_label = val_label/max_value

In [8]:
def predict():
#     srcnn_model = model()[0]
    srcnn_model = predict_model()
    srcnn_model.load_weights("checkpoints/64-9-3-5_128-64-SRCNN_model_at_epoch_999.h5")
    IMG_NAME = numpy.fromfile('ground_data/pick/recon/slice_256.b', dtype = 'uint8')
    INPUT_NAME = numpy.fromfile("sub_data/pick/recon/slice_256.b", dtype = 'uint8')
    OUTPUT_NAME = "pre2.png"

    dimension_img = int(IMG_NAME.shape[0]**(1/2))
    dimension_input = int(INPUT_NAME.shape[0]**(1/2))
    
    IMG_NAME = IMG_NAME.reshape((dimension_img, dimension_img))
    INPUT_NAME = INPUT_NAME.reshape((dimension_input, dimension_input))
    
    import cv2
    img = INPUT_NAME.copy()
    shape = IMG_NAME.shape
    Y_img = img.copy()
    Y_img = spm.imresize(Y_img, size=shape, interp='bicubic')
    bicubic = Y_img.copy()

    Y_img = Y_img.reshape((dimension_img, dimension_img, 1))
    
    for i in range(Y_img.shape[-1]):
        Y = numpy.zeros((1, shape[0], shape[1], 1), dtype=float)
        Y[0,:,:,0] = Y_img[:,:,i].astype(float) / 255.
        pre = srcnn_model.predict(Y, batch_size=1) * 255.
        
    pre[pre[:] > 255] = 255
    pre[pre[:] < 0] = 0
    pre = pre.astype(float)
    print(Y_img.shape, pre.shape)
    Y_img[6: -6, 6: -6, 0] = pre[0, :, :, 0].copy()
    cv2.imwrite(OUTPUT_NAME, Y_img)

    # psnr calculation:
    im1 = IMG_NAME
    im2 = bicubic
    im3 = cv2.imread(OUTPUT_NAME)[:,:,0]
    
    print(im1.shape, im2.shape, im3.shape)

    print ("bicubic:")
    print (cv2.PSNR(im1, im2))
    print ("SRCNN:")
    print (cv2.PSNR(im1, im3))
    
    import matplotlib.pyplot as plt
    print(im1.shape)
    plt.figure(figsize=[15,30])
    plt.subplot(311)
#     plt.imshow(im1[350:450,200:400])
    plt.title('Original')
    plt.imshow(im1)
    plt.subplot(312)
#     plt.imshow(im2[350:450,200:400])
    plt.title('Bicubic')    
    plt.imshow(im2)
    plt.subplot(313)
#     plt.imshow(im3[350:450,200:400])
    plt.title('SRCNN')
    plt.imshow(im3[10:-10,10:-10])
    plt.show()

In [ ]:
train(processing='GPU', optimizer='adam', use_data_augmentation=False, epochs=1000, lr=0.0001)

Using ADAM.
Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_1_input (InputLayer)     (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 64, 64, 1)    0           conv2d_1_input[0][0]             
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 64, 64, 1)    0           conv2d_1_input[0][0]             
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 64, 64, 1)    0           conv2d_1_input[0][0]             
_________________________

Epoch 28/1000
 - 21s - loss: 3.2046e-04 - mean_squared_error: 3.2046e-04 - val_loss: 2.5692e-05 - val_mean_squared_error: 2.5692e-05
Epoch 29/1000
 - 20s - loss: 3.2074e-04 - mean_squared_error: 3.2074e-04 - val_loss: 2.8354e-05 - val_mean_squared_error: 2.8354e-05
Epoch 30/1000
 - 20s - loss: 3.2060e-04 - mean_squared_error: 3.2060e-04 - val_loss: 2.5773e-05 - val_mean_squared_error: 2.5773e-05
Epoch 31/1000
 - 20s - loss: 3.2048e-04 - mean_squared_error: 3.2048e-04 - val_loss: 2.5709e-05 - val_mean_squared_error: 2.5709e-05
Epoch 32/1000
 - 21s - loss: 3.2032e-04 - mean_squared_error: 3.2032e-04 - val_loss: 2.6748e-05 - val_mean_squared_error: 2.6748e-05
Epoch 33/1000
 - 21s - loss: 3.2097e-04 - mean_squared_error: 3.2097e-04 - val_loss: 2.5698e-05 - val_mean_squared_error: 2.5698e-05
Epoch 34/1000
 - 21s - loss: 3.2040e-04 - mean_squared_error: 3.2040e-04 - val_loss: 2.6490e-05 - val_mean_squared_error: 2.6490e-05
Epoch 35/1000
 - 20s - loss: 3.2065e-04 - mean_squared_error: 3.2065e

Epoch 90/1000
 - 20s - loss: 3.1830e-04 - mean_squared_error: 3.1830e-04 - val_loss: 2.8319e-05 - val_mean_squared_error: 2.8319e-05
Epoch 91/1000
 - 20s - loss: 3.1891e-04 - mean_squared_error: 3.1891e-04 - val_loss: 2.5769e-05 - val_mean_squared_error: 2.5769e-05
Epoch 92/1000
 - 20s - loss: 3.1863e-04 - mean_squared_error: 3.1863e-04 - val_loss: 4.5529e-05 - val_mean_squared_error: 4.5529e-05
Epoch 93/1000
 - 20s - loss: 3.1824e-04 - mean_squared_error: 3.1824e-04 - val_loss: 2.6755e-05 - val_mean_squared_error: 2.6755e-05
Epoch 94/1000
 - 19s - loss: 3.1831e-04 - mean_squared_error: 3.1831e-04 - val_loss: 2.5658e-05 - val_mean_squared_error: 2.5658e-05
Epoch 95/1000
 - 22s - loss: 3.1821e-04 - mean_squared_error: 3.1821e-04 - val_loss: 2.7324e-05 - val_mean_squared_error: 2.7324e-05
Epoch 96/1000
 - 22s - loss: 3.1829e-04 - mean_squared_error: 3.1829e-04 - val_loss: 2.5641e-05 - val_mean_squared_error: 2.5641e-05
Epoch 97/1000
 - 22s - loss: 3.1802e-04 - mean_squared_error: 3.1802e

Epoch 152/1000
 - 23s - loss: 3.1422e-04 - mean_squared_error: 3.1422e-04 - val_loss: 2.5460e-05 - val_mean_squared_error: 2.5460e-05
Epoch 153/1000
 - 22s - loss: 3.1421e-04 - mean_squared_error: 3.1421e-04 - val_loss: 2.5809e-05 - val_mean_squared_error: 2.5809e-05
Epoch 154/1000
 - 23s - loss: 3.1386e-04 - mean_squared_error: 3.1386e-04 - val_loss: 2.6535e-05 - val_mean_squared_error: 2.6535e-05
Epoch 155/1000
 - 24s - loss: 3.1429e-04 - mean_squared_error: 3.1429e-04 - val_loss: 2.5166e-05 - val_mean_squared_error: 2.5166e-05
Epoch 156/1000
 - 22s - loss: 3.1449e-04 - mean_squared_error: 3.1449e-04 - val_loss: 2.5128e-05 - val_mean_squared_error: 2.5128e-05
Epoch 157/1000
 - 22s - loss: 3.1363e-04 - mean_squared_error: 3.1363e-04 - val_loss: 2.7786e-05 - val_mean_squared_error: 2.7786e-05
Epoch 158/1000
 - 19s - loss: 3.1395e-04 - mean_squared_error: 3.1395e-04 - val_loss: 2.6486e-05 - val_mean_squared_error: 2.6486e-05
Epoch 159/1000
 - 22s - loss: 3.1401e-04 - mean_squared_error:

Epoch 214/1000
 - 21s - loss: 3.1207e-04 - mean_squared_error: 3.1207e-04 - val_loss: 2.6707e-05 - val_mean_squared_error: 2.6707e-05
Epoch 215/1000
 - 22s - loss: 3.1214e-04 - mean_squared_error: 3.1214e-04 - val_loss: 2.4965e-05 - val_mean_squared_error: 2.4965e-05
Epoch 216/1000
 - 21s - loss: 3.1254e-04 - mean_squared_error: 3.1254e-04 - val_loss: 2.6023e-05 - val_mean_squared_error: 2.6023e-05
Epoch 217/1000
 - 22s - loss: 3.1167e-04 - mean_squared_error: 3.1167e-04 - val_loss: 2.4980e-05 - val_mean_squared_error: 2.4980e-05
Epoch 218/1000
 - 22s - loss: 3.1213e-04 - mean_squared_error: 3.1213e-04 - val_loss: 2.4919e-05 - val_mean_squared_error: 2.4919e-05
Epoch 219/1000
 - 21s - loss: 3.1215e-04 - mean_squared_error: 3.1215e-04 - val_loss: 2.4771e-05 - val_mean_squared_error: 2.4771e-05
Epoch 220/1000
 - 22s - loss: 3.1188e-04 - mean_squared_error: 3.1188e-04 - val_loss: 2.5677e-05 - val_mean_squared_error: 2.5677e-05
Epoch 221/1000
 - 21s - loss: 3.1186e-04 - mean_squared_error:

...

Epoch 710/1000
 - 22s - loss: 3.0922e-04 - mean_squared_error: 3.0922e-04 - val_loss: 2.4663e-05 - val_mean_squared_error: 2.4663e-05
Epoch 711/1000
 - 22s - loss: 3.0912e-04 - mean_squared_error: 3.0912e-04 - val_loss: 2.4545e-05 - val_mean_squared_error: 2.4545e-05
Epoch 712/1000
 - 20s - loss: 3.0920e-04 - mean_squared_error: 3.0920e-04 - val_loss: 2.4733e-05 - val_mean_squared_error: 2.4733e-05
Epoch 713/1000
 - 22s - loss: 3.0907e-04 - mean_squared_error: 3.0907e-04 - val_loss: 2.4947e-05 - val_mean_squared_error: 2.4947e-05
Epoch 714/1000
 - 22s - loss: 3.0906e-04 - mean_squared_error: 3.0906e-04 - val_loss: 2.7637e-05 - val_mean_squared_error: 2.7637e-05
Epoch 715/1000
 - 24s - loss: 3.0920e-04 - mean_squared_error: 3.0920e-04 - val_loss: 2.4554e-05 - val_mean_squared_error: 2.4554e-05
Epoch 716/1000
 - 21s - loss: 3.0904e-04 - mean_squared_error: 3.0904e-04 - val_loss: 2.4799e-05 - val_mean_squared_error: 2.4799e-05
Epoch 717/1000
 - 20s - loss: 3.0914e-04 - mean_squared_error:

Epoch 772/1000
 - 21s - loss: 3.0917e-04 - mean_squared_error: 3.0917e-04 - val_loss: 2.5501e-05 - val_mean_squared_error: 2.5501e-05
Epoch 773/1000
 - 20s - loss: 3.0898e-04 - mean_squared_error: 3.0898e-04 - val_loss: 2.4894e-05 - val_mean_squared_error: 2.4894e-05
Epoch 774/1000
 - 21s - loss: 3.0890e-04 - mean_squared_error: 3.0890e-04 - val_loss: 2.4651e-05 - val_mean_squared_error: 2.4651e-05
Epoch 775/1000
 - 21s - loss: 3.0913e-04 - mean_squared_error: 3.0913e-04 - val_loss: 2.5052e-05 - val_mean_squared_error: 2.5052e-05
Epoch 776/1000
 - 20s - loss: 3.0896e-04 - mean_squared_error: 3.0896e-04 - val_loss: 2.4689e-05 - val_mean_squared_error: 2.4689e-05
Epoch 777/1000
 - 20s - loss: 3.0895e-04 - mean_squared_error: 3.0895e-04 - val_loss: 2.4632e-05 - val_mean_squared_error: 2.4632e-05
Epoch 778/1000
 - 23s - loss: 3.0893e-04 - mean_squared_error: 3.0893e-04 - val_loss: 2.4662e-05 - val_mean_squared_error: 2.4662e-05
Epoch 779/1000
 - 22s - loss: 3.0895e-04 - mean_squared_error:

Epoch 834/1000
 - 19s - loss: 3.0902e-04 - mean_squared_error: 3.0902e-04 - val_loss: 2.4831e-05 - val_mean_squared_error: 2.4831e-05
Epoch 835/1000
 - 24s - loss: 3.0875e-04 - mean_squared_error: 3.0875e-04 - val_loss: 2.5204e-05 - val_mean_squared_error: 2.5204e-05
Epoch 836/1000
 - 21s - loss: 3.0886e-04 - mean_squared_error: 3.0886e-04 - val_loss: 2.4919e-05 - val_mean_squared_error: 2.4919e-05
Epoch 837/1000
 - 22s - loss: 3.0887e-04 - mean_squared_error: 3.0887e-04 - val_loss: 2.5914e-05 - val_mean_squared_error: 2.5914e-05
Epoch 838/1000
 - 23s - loss: 3.0874e-04 - mean_squared_error: 3.0874e-04 - val_loss: 2.5275e-05 - val_mean_squared_error: 2.5275e-05
Epoch 839/1000
 - 22s - loss: 3.0886e-04 - mean_squared_error: 3.0886e-04 - val_loss: 2.4927e-05 - val_mean_squared_error: 2.4927e-05
Epoch 840/1000
 - 22s - loss: 3.0893e-04 - mean_squared_error: 3.0893e-04 - val_loss: 2.4623e-05 - val_mean_squared_error: 2.4623e-05
Epoch 841/1000
 - 24s - loss: 3.0881e-04 - mean_squared_error:

Epoch 896/1000
 - 20s - loss: 3.0874e-04 - mean_squared_error: 3.0874e-04 - val_loss: 2.4980e-05 - val_mean_squared_error: 2.4980e-05
Epoch 897/1000
 - 19s - loss: 3.0873e-04 - mean_squared_error: 3.0873e-04 - val_loss: 2.4889e-05 - val_mean_squared_error: 2.4889e-05
Epoch 898/1000
 - 20s - loss: 3.0881e-04 - mean_squared_error: 3.0881e-04 - val_loss: 2.4704e-05 - val_mean_squared_error: 2.4704e-05
Epoch 899/1000
 - 24s - loss: 3.0880e-04 - mean_squared_error: 3.0880e-04 - val_loss: 2.5534e-05 - val_mean_squared_error: 2.5534e-05
Epoch 900/1000
 - 20s - loss: 3.0856e-04 - mean_squared_error: 3.0856e-04 - val_loss: 2.5297e-05 - val_mean_squared_error: 2.5297e-05
Epoch 901/1000
 - 23s - loss: 3.0875e-04 - mean_squared_error: 3.0875e-04 - val_loss: 2.7012e-05 - val_mean_squared_error: 2.7012e-05
Epoch 902/1000
 - 21s - loss: 3.0872e-04 - mean_squared_error: 3.0872e-04 - val_loss: 2.4611e-05 - val_mean_squared_error: 2.4611e-05
Epoch 903/1000
 - 22s - loss: 3.0863e-04 - mean_squared_error:

Epoch 958/1000
 - 21s - loss: 3.0871e-04 - mean_squared_error: 3.0871e-04 - val_loss: 2.4595e-05 - val_mean_squared_error: 2.4595e-05
Epoch 959/1000
 - 23s - loss: 3.0847e-04 - mean_squared_error: 3.0847e-04 - val_loss: 2.4755e-05 - val_mean_squared_error: 2.4755e-05
Epoch 960/1000
 - 22s - loss: 3.0867e-04 - mean_squared_error: 3.0867e-04 - val_loss: 2.4912e-05 - val_mean_squared_error: 2.4912e-05
Epoch 961/1000
 - 23s - loss: 3.0900e-04 - mean_squared_error: 3.0900e-04 - val_loss: 2.4717e-05 - val_mean_squared_error: 2.4717e-05
Epoch 962/1000
 - 23s - loss: 3.0844e-04 - mean_squared_error: 3.0844e-04 - val_loss: 2.4594e-05 - val_mean_squared_error: 2.4594e-05
Epoch 963/1000
 - 23s - loss: 3.0866e-04 - mean_squared_error: 3.0866e-04 - val_loss: 2.4695e-05 - val_mean_squared_error: 2.4695e-05
Epoch 964/1000
 - 20s - loss: 3.0867e-04 - mean_squared_error: 3.0867e-04 - val_loss: 2.5067e-05 - val_mean_squared_error: 2.5067e-05
Epoch 965/1000
 - 23s - loss: 3.0849e-04 - mean_squared_error:

In [ ]:
predict()

In [ ]:
a = mpimg.imread('butterfly_GT.bmp')
plt.imshow(a)
del(a)